# Systematic Macro Credit — End-to-End Workflow

This notebook demonstrates the complete workflow for evaluating a single CDX signal:

1. **Data Fetching**: Fetch data from Bloomberg Terminal (or generate synthetic fallback)
2. **Data Loading**: Load and validate market data
3. **Signal Computation**: Calculate cdx_etf_basis signal
4. **Signal Evaluation**: Assess signal-product suitability (pre-backtest quality gate)
5. **Backtesting**: Run strategy backtest with position tracking (if evaluation passes)
6. **Visualization**: Analyze results with interactive plots

Bloomberg Terminal is used by default when available, with graceful fallback to synthetic data.

## 1. Setup and Imports

In [37]:
import logging
import sys
from pathlib import Path

# Configure logging for notebook (not in library code)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import project modules
import aponyx
from aponyx.data import fetch_cdx, fetch_vix, fetch_etf
from aponyx.data.sample_data import (
    generate_cdx_sample,
    generate_etf_sample,
    generate_vix_sample,
)
from aponyx.models import (
    SignalRegistry,
    SignalConfig,
    compute_registered_signals,
)
from aponyx.evaluation.suitability import (
    evaluate_signal_suitability,
    SuitabilityConfig,
    generate_suitability_report,
)
from aponyx.backtest.config import BacktestConfig
from aponyx.backtest.engine import run_backtest
from aponyx.backtest.metrics import compute_performance_metrics
from aponyx.visualization.plots import (
    plot_signal,
    plot_equity_curve,
    plot_drawdown,
)

# Global flag to track data source
using_bloomberg = False

print("Imports successful")
print(f"Framework version: {aponyx.__version__}")

Imports successful
Framework version: 0.1.4


## 2. Fetch Market Data

Fetch data from Bloomberg Terminal (primary) or generate synthetic fallback:
- **CDX IG 5Y**: Investment grade credit spreads
- **HYG ETF**: High yield credit ETF
- **VIX**: Volatility index

In [38]:
# Configuration
START_DATE = "2024-01-01"
SEED = 42

print("Attempting Bloomberg Terminal connection...")
try:
    from aponyx.data import BloombergSource
    
    source = BloombergSource()
    logger.info("Bloomberg Terminal connection established")
    print("✓ Bloomberg Terminal available")
    print(f"  Fetching data from {START_DATE} to present...")
    using_bloomberg = True
    
    # Fetch from Bloomberg
    cdx_data = fetch_cdx(source, security="cdx_ig_5y", start_date=START_DATE)
    etf_data = fetch_etf(source, security="hyg", start_date=START_DATE)
    vix_data = fetch_vix(source, start_date=START_DATE)
    
    logger.info("Fetched market data from Bloomberg: %d days", len(cdx_data))
    print(f"✓ Fetched {len(cdx_data)} days of market data from Bloomberg")
    print(f"  CDX IG 5Y: {len(cdx_data)} rows")
    print(f"  HYG ETF: {len(etf_data)} rows")
    print(f"  VIX: {len(vix_data)} rows")
    
    # Rename columns for consistency with downstream code
    cdx_data = cdx_data[["spread"]].copy()
    etf_data = etf_data[["spread"]].copy()
    vix_data = vix_data[["level"]].copy()
    
except (ImportError, ModuleNotFoundError) as e:
    logger.warning("Bloomberg Terminal not available: missing xbbg or blpapi module")
    print("! Bloomberg Terminal not installed")
    print(f"  Reason: {e}")
    print("\n  Falling back to synthetic data...")
    using_bloomberg = False
    
    PERIODS = 252  # One year of trading days
    START_DATE = "2023-01-01"
    
    cdx_data = generate_cdx_sample(
        start_date=START_DATE,
        periods=PERIODS,
        index_name="CDX_IG",
        tenor="5Y",
        base_spread=70.0,
        volatility=3.0,
        seed=SEED,
    )
    
    etf_data = generate_etf_sample(
        start_date=START_DATE,
        periods=PERIODS,
        ticker="HYG",
        base_price=75.0,
        volatility=0.6,
        seed=SEED + 1,
    )
    
    vix_data = generate_vix_sample(
        start_date=START_DATE,
        periods=PERIODS,
        base_vix=15.0,
        volatility=2.0,
        seed=SEED + 2,
    )
    
    logger.info("Generated %d days of synthetic data", len(cdx_data))
    print(f"✓ Generated {len(cdx_data)} days of synthetic data")
    
    # Display sample data
    data_source = "Bloomberg Terminal" if using_bloomberg else "Synthetic data"
    print(f"\nData source: {data_source}")
    print("\nCDX Sample:")
    print(cdx_data.head())
    print("\nETF Sample:")
    print(etf_data.head())
    print("\nVIX Sample:")
    print(vix_data.head())
    
except BaseException as e:
    # Catch pytest.Skipped and other xbbg errors
    error_str = str(e).lower()
    error_type = str(type(e).__name__).lower()
    if "blpapi" in error_str or "could not import" in error_str or "skipped" in error_type:
        logger.warning("Bloomberg Terminal not available: blpapi module missing")
        print("! Bloomberg Terminal not installed")
    else:
        logger.warning("Bloomberg Terminal connection failed: %s", e)
        print("! Bloomberg Terminal not running or authentication failed")
        print(f"  Reason: {type(e).__name__}: {e}")
    
    print("\n  Falling back to synthetic data...")
    using_bloomberg = False
    
    PERIODS = 252  # One year of trading days
    START_DATE = "2023-01-01"
    
    cdx_data = generate_cdx_sample(
        start_date=START_DATE,
        periods=PERIODS,
        index_name="CDX_IG",
        tenor="5Y",
        base_spread=70.0,
        volatility=3.0,
        seed=SEED,
    )
    
    etf_data = generate_etf_sample(
        start_date=START_DATE,
        periods=PERIODS,
        ticker="HYG",
        base_price=75.0,
        volatility=0.6,
        seed=SEED + 1,
    )
    
    vix_data = generate_vix_sample(
        start_date=START_DATE,
        periods=PERIODS,
        base_vix=15.0,
        volatility=2.0,
        seed=SEED + 2,
    )
    
    logger.info("Generated %d days of synthetic data", len(cdx_data))
    print(f"✓ Generated {len(cdx_data)} days of synthetic data")
    
    # Display sample data
    data_source = "Bloomberg Terminal" if using_bloomberg else "Synthetic data"
    print(f"\nData source: {data_source}")
    print("\nCDX Sample:")
    print(cdx_data.head())
    print("\nETF Sample:")
    print(etf_data.head())
    print("\nVIX Sample:")
    print(vix_data.head())

2025-11-07 22:49:21,665 - __main__ - INFO - Bloomberg Terminal connection established
2025-11-07 22:49:21,666 - aponyx.data.fetch - INFO - Fetching CDX from bloomberg
2025-11-07 22:49:21,667 - aponyx.data.providers.bloomberg - INFO - Fetching cdx from Bloomberg: ticker=CDX IG CDSI GEN 5Y Corp, dates=2024-01-01 to 2025-11-07
2025-11-07 22:49:21,669 - __main__ - WARNING - Bloomberg Terminal not available: blpapi module missing
2025-11-07 22:49:21,666 - aponyx.data.fetch - INFO - Fetching CDX from bloomberg
2025-11-07 22:49:21,667 - aponyx.data.providers.bloomberg - INFO - Fetching cdx from Bloomberg: ticker=CDX IG CDSI GEN 5Y Corp, dates=2024-01-01 to 2025-11-07
2025-11-07 22:49:21,669 - __main__ - WARNING - Bloomberg Terminal not available: blpapi module missing
2025-11-07 22:49:21,670 - aponyx.data.sample_data - INFO - Generating CDX sample: index=CDX_IG, tenor=5Y, periods=252
2025-11-07 22:49:21,672 - aponyx.data.sample_data - INFO - Generating ETF sample: ticker=HYG, periods=252
2025

Attempting Bloomberg Terminal connection...
✓ Bloomberg Terminal available
  Fetching data from 2024-01-01 to present...
! Bloomberg Terminal not installed

  Falling back to synthetic data...
✓ Generated 252 days of synthetic data

Data source: Synthetic data

CDX Sample:
        date     spread      index tenor  series
0 2023-01-01  70.000000  CDX_IG_5Y    5Y      42
1 2023-01-02  70.914151  CDX_IG_5Y    5Y      42
2 2023-01-03  67.702784  CDX_IG_5Y    5Y      42
3 2023-01-04  70.183859  CDX_IG_5Y    5Y      42
4 2023-01-05  72.987167  CDX_IG_5Y    5Y      42

ETF Sample:
        date     spread ticker
0 2023-01-01  75.154196    HYG
1 2023-01-02  75.570604    HYG
2 2023-01-03  75.224964    HYG
3 2023-01-04  74.687576    HYG
4 2023-01-05  73.514234    HYG

VIX Sample:
        date      level
0 2023-01-01  15.000000
1 2023-01-02  15.204610
2 2023-01-03  17.446844
3 2023-01-04  18.237730
4 2023-01-05  15.714323


## 3. Merge Market Data

Use standard pandas operations to combine data sources.

In [39]:
# Prepare data for merging based on data source
if using_bloomberg:
    # Bloomberg data already has DatetimeIndex and proper column names
    cdx_prep = cdx_data[["spread"]].rename(columns={"spread": "cdx_spread"})
    vix_prep = vix_data[["level"]].rename(columns={"level": "vix_level"})
    etf_prep = etf_data[["spread"]].rename(columns={"spread": "etf_spread"})
    
    # Merge on index (date)
    market_data = cdx_prep.join(vix_prep).join(etf_prep)
else:
    # Synthetic data has 'date' column
    cdx_prep = cdx_data[["date", "spread"]].rename(columns={"spread": "cdx_spread"})
    vix_prep = vix_data[["date", "level"]].rename(columns={"level": "vix_level"})
    etf_prep = etf_data[["date", "spread"]].rename(columns={"spread": "etf_spread"})
    
    # Merge on date
    market_data = cdx_prep.merge(vix_prep, on="date").merge(etf_prep, on="date")
    market_data = market_data.set_index("date")

print(f"Merged market data: {len(market_data)} rows")
print(f"Columns: {list(market_data.columns)}")
print(f"Date range: {market_data.index.min().date()} to {market_data.index.max().date()}")
print("\nMerged Data Sample:")
print(market_data.head())

Merged market data: 252 rows
Columns: ['cdx_spread', 'vix_level', 'etf_spread']
Date range: 2023-01-01 to 2023-09-09

Merged Data Sample:
            cdx_spread  vix_level  etf_spread
date                                         
2023-01-01   70.000000  15.000000   75.154196
2023-01-02   70.914151  15.204610   75.570604
2023-01-03   67.702784  17.446844   75.224964
2023-01-04   70.183859  18.237730   74.687576
2023-01-05   72.987167  15.714323   73.514234


## 4. Compute Signals via Registry

Calculate signals using SignalRegistry for governance and reproducibility.

**Signal Convention**: Positive = Long credit risk (buy CDX/sell protection)

In [40]:
# Initialize signal registry
signal_catalog = Path("../models/signal_catalog.json")
signal_registry = SignalRegistry(signal_catalog)

print(f"Loaded signal registry: {signal_catalog}")
print(f"Enabled signals: {list(signal_registry.get_enabled().keys())}")

# Configure signal parameters
signal_config = SignalConfig(
    lookback=20,
    min_periods=10,
)

# Prepare market data dict for registry
market_data_dict = {
    "cdx": market_data[["cdx_spread"]].rename(columns={"cdx_spread": "spread"}),
    "vix": market_data[["vix_level"]].rename(columns={"vix_level": "level"}),
    "etf": market_data[["etf_spread"]].rename(columns={"etf_spread": "spread"}),
}

# Compute all signals via registry
signals = compute_registered_signals(signal_registry, market_data_dict, signal_config)

print(f"\nComputed {len(signals)} signals:")
for name, signal in signals.items():
    valid = signal.notna().sum()
    print(f"  {name}: {valid} valid observations")

# Add signals to market data for analysis
for name, signal in signals.items():
    market_data[name] = signal

# Use cdx_etf_basis for backtest
signal = signals["cdx_etf_basis"]

print("\ncdx_etf_basis Signal Statistics:")
print(market_data[["cdx_etf_basis"]].describe())

2025-11-07 22:49:21,699 - aponyx.models.registry - INFO - Loaded signal registry: catalog=..\models\signal_catalog.json, signals=3, enabled=3
2025-11-07 22:49:21,702 - aponyx.models.catalog - INFO - Computing 3 enabled signals: cdx_etf_basis, cdx_vix_gap, spread_momentum
2025-11-07 22:49:21,703 - aponyx.models.signals - INFO - Computing CDX-ETF basis: cdx_rows=252, etf_rows=252, lookback=20
2025-11-07 22:49:21,704 - aponyx.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=252, vix_rows=252, lookback=20
2025-11-07 22:49:21,706 - aponyx.models.signals - INFO - Computing spread momentum: cdx_rows=252, lookback=20
2025-11-07 22:49:21,702 - aponyx.models.catalog - INFO - Computing 3 enabled signals: cdx_etf_basis, cdx_vix_gap, spread_momentum
2025-11-07 22:49:21,703 - aponyx.models.signals - INFO - Computing CDX-ETF basis: cdx_rows=252, etf_rows=252, lookback=20
2025-11-07 22:49:21,704 - aponyx.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=252, vix_rows=252, lookback=20
2025

Loaded signal registry: ..\models\signal_catalog.json
Enabled signals: ['cdx_etf_basis', 'cdx_vix_gap', 'spread_momentum']

Computed 3 signals:
  cdx_etf_basis: 243 valid observations
  cdx_vix_gap: 234 valid observations
  spread_momentum: 232 valid observations

cdx_etf_basis Signal Statistics:
       cdx_etf_basis
count     243.000000
mean        0.025677
std         1.181586
min        -2.628056
25%        -0.907645
50%         0.015730
75%         0.825891
max         3.123916


## 5. Visualize Signal

Plot cdx_etf_basis signal to understand strategy driver.

In [41]:
# Plot signal
fig_signal = plot_signal(
    signal=market_data["cdx_etf_basis"],
    title="CDX-ETF Basis Signal",
    threshold_lines=[1.5, -1.5],
)
fig_signal.show()

print("Signal visualization complete")

2025-11-07 22:49:21,716 - aponyx.visualization.plots - INFO - Plotting signal: 252 observations


Signal visualization complete


## 6. Evaluate Signal-Product Suitability

Pre-backtest quality gate: assess whether the signal is suitable for this product.

**Evaluation Framework:**
- **Data Health** (20%): Sample size and missing data
- **Predictive Power** (40%): Forward-looking correlation
- **Economic Significance** (20%): Return predictability
- **Stability** (20%): Sign consistency across subperiods

**Decision Thresholds:**
- **PASS** (≥ 0.7): Proceed to backtest
- **HOLD** (0.4-0.7): Marginal quality, requires manual review
- **FAIL** (< 0.4): Do not backtest

In [ ]:
# Configure evaluation
suitability_config = SuitabilityConfig(
    lags=[1, 3, 5],  # Forward-looking lags for predictive power
    min_obs=100,  # Minimum observations required (reduced for demo data)
)

# Evaluate signal-product suitability
evaluation_result = evaluate_signal_suitability(
    signal=market_data["cdx_etf_basis"],
    target_change=market_data["cdx_spread"],
    config=suitability_config,
)

# Display evaluation results
print("="*60)
print("SIGNAL SUITABILITY EVALUATION")
print("="*60)
print(f"\nSignal: cdx_etf_basis")
print(f"Product: CDX IG 5Y spreads")
print(f"\nComposite Score: {evaluation_result.composite_score:.3f}")
print(f"Decision: {evaluation_result.decision}")
print(f"\nComponent Scores:")
print(f"  Data Health:      {evaluation_result.data_health_score:.3f} (20% weight)")
print(f"  Predictive Power: {evaluation_result.predictive_score:.3f} (40% weight)")
print(f"  Economic Signif.: {evaluation_result.economic_score:.3f} (20% weight)")
print(f"  Stability:        {evaluation_result.stability_score:.3f} (20% weight)")
print("\nKey Metrics:")
print(f"  Valid observations: {evaluation_result.valid_obs}")
print(f"  Missing data: {evaluation_result.missing_pct:.1f}%")
print(f"  Effect size: {evaluation_result.effect_size_bps:.2f} bps per 1σ signal")

# Generate markdown report
report_dir = Path("reports/suitability")
report_dir.mkdir(parents=True, exist_ok=True)
report_path = report_dir / "cdx_etf_basis_evaluation.md"

report_content = generate_suitability_report(
    result=evaluation_result,
    signal_id="cdx_etf_basis",
    product_id="cdx_ig_5y",
)

with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_content)

print(f"\nEvaluation report saved to: {report_path}")

# Decision gate
if evaluation_result.decision == "FAIL":
    print("\n⚠️  Signal evaluation FAILED - Skipping backtest")
    print("    Signal quality is insufficient for this product.")
    print("    Consider:")
    print("      - Using different signal parameters")
    print("      - Evaluating with a different product")
    print("      - Investigating data quality issues")
elif evaluation_result.decision == "HOLD":
    print("\n⚠️  Signal evaluation HOLD - Proceed with caution")
    print("    Signal shows marginal quality. Manual review recommended.")
else:
    print(f"\n✓ Signal evaluation PASSED (score: {evaluation_result.composite_score:.3f})")
    print("  Proceeding to backtest...")

2025-11-07 22:49:21,777 - aponyx.evaluation.suitability.evaluator - INFO - Starting suitability evaluation: signal=cdx_etf_basis, config=SuitabilityConfig(lags=[1, 3, 5], min_obs=100, pass_threshold=0.7, hold_threshold=0.4, data_health_weight=0.2, predictive_weight=0.4, economic_weight=0.2, stability_weight=0.2)
2025-11-07 22:49:21,778 - aponyx.evaluation.suitability.evaluator - INFO - Data health score: 0.821
2025-11-07 22:49:21,785 - aponyx.evaluation.suitability.evaluator - INFO - Predictive score: 1.000 (mean |t-stat|=7.660 across 3 lags)
2025-11-07 22:49:21,778 - aponyx.evaluation.suitability.evaluator - INFO - Data health score: 0.821
2025-11-07 22:49:21,785 - aponyx.evaluation.suitability.evaluator - INFO - Predictive score: 1.000 (mean |t-stat|=7.660 across 3 lags)
2025-11-07 22:49:21,786 - aponyx.evaluation.suitability.evaluator - INFO - Economic score: 1.000
2025-11-07 22:49:21,787 - aponyx.evaluation.suitability.evaluator - INFO - Stability score: 1.000
2025-11-07 22:49:21,7

SIGNAL SUITABILITY EVALUATION

Signal: cdx_etf_basis
Product: CDX IG 5Y spreads

Composite Score: 0.964
Decision: PASS

Component Scores:
  Data Health:      0.821 (20% weight)
  Predictive Power: 1.000 (40% weight)
  Economic Signif.: 1.000 (20% weight)
  Stability:        1.000 (20% weight)

Key Metrics:
  Valid observations: 243
  Missing data: 3.6%
  Effect size: 2.42 bps per 1σ signal

Evaluation report saved to: reports\suitability\cdx_etf_basis_evaluation.md

✓ Signal evaluation PASSED (score: 0.964)
  Proceeding to backtest...


## 7. Run Backtest (Conditional on Evaluation)

Execute strategy backtest with position limits and transaction costs.

**Note:** Backtest only runs if evaluation result is PASS or HOLD.

In [43]:
# Only run backtest if evaluation passes quality gate
if evaluation_result.decision in ["PASS", "HOLD"]:
    # Configure backtest
    backtest_config = BacktestConfig(
        entry_threshold=1.5,
        exit_threshold=0.75,
        position_size=10.0,  # $10M notional
        transaction_cost_bps=1.0,  # 1 bps
        max_holding_days=None,  # No time limit
        dv01_per_million=4750.0,
    )

    # Run backtest
    backtest_result = run_backtest(
        composite_signal=market_data["cdx_etf_basis"],
        spread=market_data["cdx_spread"],
        config=backtest_config,
    )

    print("Backtest complete")
    print(f"\nBacktest Results: {len(backtest_result.positions)} periods")
    print("\nPosition Data Sample:")
    print(backtest_result.positions.head())
    print("\nP&L Data Sample:")
    print(backtest_result.pnl.head())
    print(f"\nFinal Cumulative P&L: ${backtest_result.pnl['cumulative_pnl'].iloc[-1]:,.2f}")
else:
    print("\n⚠️  Backtest skipped due to FAIL evaluation result")
    backtest_result = None

2025-11-07 22:49:21,797 - aponyx.backtest.engine - INFO - Starting backtest: dates=252, entry_threshold=1.50, position_size=10.0MM
2025-11-07 22:49:21,807 - aponyx.backtest.engine - INFO - Backtest complete: trades=15, total_pnl=$-1023477, avg_per_trade=$-68232


Backtest complete

Backtest Results: 243 periods

Position Data Sample:
              signal  position  days_held     spread
date                                                
2023-01-10 -0.909162         0          0  64.482197
2023-01-11 -1.426287         0          0  62.474845
2023-01-12 -0.433645         0          0  65.865555
2023-01-13  0.442488         0          0  68.612375
2023-01-14  0.577850         0          0  68.949230

P&L Data Sample:
            spread_pnl  cost  net_pnl  cumulative_pnl
date                                                 
2023-01-10         0.0   0.0      0.0             0.0
2023-01-11         0.0   0.0      0.0             0.0
2023-01-12         0.0   0.0      0.0             0.0
2023-01-13         0.0   0.0      0.0             0.0
2023-01-14         0.0   0.0      0.0             0.0

Final Cumulative P&L: $-1,023,476.72


## 8. Calculate Performance Metrics

Compute Sharpe ratio, max drawdown, and other key statistics.

**Note:** Only computed if backtest was executed.

In [44]:
# Calculate performance metrics (only if backtest ran)
if backtest_result is not None:
    metrics = compute_performance_metrics(
        pnl_df=backtest_result.pnl,
        positions_df=backtest_result.positions,
    )

    print("Performance metrics calculated")
    print("\n" + "="*50)
    print("PERFORMANCE SUMMARY")
    print("="*50)
    print(f"Total Return:        ${metrics.total_return:>12,.2f}")
    print(f"Sharpe Ratio:        {metrics.sharpe_ratio:>12.2f}")
    print(f"Sortino Ratio:       {metrics.sortino_ratio:>12.2f}")
    print(f"Max Drawdown:        ${metrics.max_drawdown:>12,.2f}")
    print(f"Calmar Ratio:        {metrics.calmar_ratio:>12.2f}")
    print(f"Hit Rate:            {metrics.hit_rate:>12.1%}")
    print(f"Number of Trades:    {metrics.n_trades:>12d}")
    print(f"Avg Holding Days:    {metrics.avg_holding_days:>12.1f}")
    print(f"Avg Win:             ${metrics.avg_win:>12,.2f}")
    print(f"Avg Loss:            ${metrics.avg_loss:>12,.2f}")
    print(f"Win/Loss Ratio:      {metrics.win_loss_ratio:>12.2f}")
    print(f"Annualized Return:   ${metrics.annualized_return:>12,.2f}")
    print(f"Annual Volatility:   ${metrics.annualized_volatility:>12,.2f}")
    print("="*50)
else:
    print("\n⚠️  Performance metrics skipped (no backtest results)")
    metrics = None

2025-11-07 22:49:21,817 - aponyx.backtest.metrics - INFO - Computing performance metrics
2025-11-07 22:49:21,821 - aponyx.backtest.metrics - INFO - Metrics computed: sharpe=-0.42, max_dd=$-6394323, hit_rate=33.3%


Performance metrics calculated

PERFORMANCE SUMMARY
Total Return:        $-1,023,476.72
Sharpe Ratio:               -0.42
Sortino Ratio:              -0.44
Max Drawdown:        $-6,394,322.82
Calmar Ratio:               -0.17
Hit Rate:                   33.3%
Number of Trades:              15
Avg Holding Days:             3.2
Avg Win:             $  931,968.94
Avg Loss:            $ -777,821.39
Win/Loss Ratio:              1.20
Annualized Return:   $-1,061,383.26
Annual Volatility:   $2,512,185.14


## 9. Visualize Results

Create interactive plots for equity curve and drawdown analysis.

**Note:** Visualizations only generated if backtest was executed.

In [45]:
if backtest_result is not None:
    import plotly.express as px

    # Plot positions over time (simple line chart)
    fig_positions = px.line(
        backtest_result.positions,
        y="position",
        title="Position History",
        labels={"position": "Position (+1=Long, -1=Short, 0=Flat)", "date": "Date"},
    )
    fig_positions.show()

    print("Position visualization complete")
else:
    print("\n⚠️  Position visualization skipped (no backtest results)")

Position visualization complete


In [46]:
if backtest_result is not None:
    # Plot P&L evolution (equity curve)
    fig_pnl = plot_equity_curve(
        pnl=backtest_result.pnl["net_pnl"],
        title="Cumulative P&L",
        show_drawdown_shading=True,
    )
    fig_pnl.show()

    print("P&L visualization complete")
else:
    print("\n⚠️  P&L visualization skipped (no backtest results)")

2025-11-07 22:49:21,867 - aponyx.visualization.plots - INFO - Plotting equity curve: 243 observations


P&L visualization complete


In [47]:
if backtest_result is not None:
    # Plot drawdown analysis
    fig_drawdown = plot_drawdown(
        pnl=backtest_result.pnl["net_pnl"],
        title="Drawdown Analysis",
        show_underwater_chart=True,
    )
    fig_drawdown.show()

    print("Drawdown visualization complete")
else:
    print("\n⚠️  Drawdown visualization skipped (no backtest results)")

2025-11-07 22:49:21,939 - aponyx.visualization.plots - INFO - Plotting drawdown: 243 observations


Drawdown visualization complete


## 10. Save Results

Save evaluation report, backtest results, and metrics for further analysis.

In [48]:
from aponyx.persistence.parquet_io import save_parquet

# Create output directory
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save evaluation report (already saved above)
print(f"Evaluation report: {report_path}")

# Save backtest results (only if backtest ran)
if backtest_result is not None:
    positions_path = output_dir / "backtest_positions.parquet"
    save_parquet(backtest_result.positions, positions_path)

    pnl_path = output_dir / "backtest_pnl.parquet"
    save_parquet(backtest_result.pnl, pnl_path)

    print(f"\nBacktest results saved to {output_dir}:")
    print(f"   - Positions: {positions_path}")
    print(f"   - P&L: {pnl_path}")
else:
    print("\n⚠️  Backtest results not saved (evaluation failed)")

print(f"\nPerformance metrics will be saved with run metadata in the next section.")

2025-11-07 22:49:21,997 - aponyx.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_positions.parquet, rows=243, columns=4, compression=snappy
2025-11-07 22:49:22,000 - aponyx.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_pnl.parquet, rows=243, columns=4, compression=snappy


Evaluation report: reports\suitability\cdx_etf_basis_evaluation.md

Backtest results saved to data\processed:
   - Positions: data\processed\backtest_positions.parquet
   - P&L: data\processed\backtest_pnl.parquet

Performance metrics will be saved with run metadata in the next section.


## 11. Governance and Metadata Tracking

Demonstrate metadata tracking and version control for reproducibility.

In [ ]:
# Track comprehensive metadata for reproducibility
from datetime import datetime
from dataclasses import asdict
from aponyx.persistence.json_io import save_json

run_metadata = {
    "run_id": f"end_to_end_demo_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    "timestamp": datetime.now().isoformat(),
    "framework_version": aponyx.__version__,
    "data_source": "bloomberg" if using_bloomberg else "synthetic",
    "signal_config": {
        "lookback": signal_config.lookback,
        "min_periods": signal_config.min_periods,
    },
    "suitability_config": {
        "lags": suitability_config.lags,
        "min_obs": suitability_config.min_obs,
    },
    "evaluation_result": {
        "composite_score": evaluation_result.composite_score,
        "decision": evaluation_result.decision,
        "data_health_score": evaluation_result.data_health_score,
        "predictive_score": evaluation_result.predictive_score,
        "economic_score": evaluation_result.economic_score,
        "stability_score": evaluation_result.stability_score,
    },
    "backtest_config": {
        "entry_threshold": backtest_config.entry_threshold,
        "exit_threshold": backtest_config.exit_threshold,
        "position_size": backtest_config.position_size,
        "transaction_cost_bps": backtest_config.transaction_cost_bps,
        "dv01_per_million": backtest_config.dv01_per_million,
    } if backtest_result is not None else None,
    "data_info": {
        "start_date": START_DATE,
        "periods": len(market_data),
        "seed": SEED if not using_bloomberg else None,
    },
    "performance_metrics": asdict(metrics) if metrics is not None else None,
}

# Save comprehensive metadata with all metrics
metadata_output = output_dir / "run_metadata.json"
save_json(run_metadata, metadata_output)

data_source_str = "Bloomberg Terminal" if using_bloomberg else "Synthetic data"
print("Governance Metadata Saved:")
print(f"  Run ID: {run_metadata['run_id']}")
print(f"  Framework version: {run_metadata['framework_version']}")
print(f"  Data source: {data_source_str}")
print(f"  Timestamp: {run_metadata['timestamp']}")
print(f"  Metadata file: {metadata_output}")

print("\nReproducibility Information:")
print(f"  Signal config: lookback={signal_config.lookback}, min_periods={signal_config.min_periods}")
print(f"  Evaluation config: lags={suitability_config.lags}, min_obs={suitability_config.min_obs}")
print(f"  Data: {len(market_data)} days starting {START_DATE}")
if not using_bloomberg:
    print(f"  Synthetic data seed: {SEED}")
if backtest_result is not None:
    print(f"  Thresholds: entry={backtest_config.entry_threshold}, exit={backtest_config.exit_threshold}")

print("\nEvaluation Results (included in metadata):")
print(f"  Composite Score: {evaluation_result.composite_score:.3f}")
print(f"  Decision: {evaluation_result.decision}")

if metrics is not None:
    print("\nPerformance Metrics (included in metadata):")
    print(f"  Sharpe Ratio: {metrics.sharpe_ratio:.3f}")
    print(f"  Total Return: ${metrics.total_return:,.2f}")
    print(f"  Max Drawdown: ${metrics.max_drawdown:,.2f}")
    print(f"  Number of Trades: {metrics.n_trades}")
    print(f"  Hit Rate: {metrics.hit_rate:.1%}")

print("\nThis metadata enables:")
print("  ✓ Configuration documentation")
print("  ✓ Evaluation tracking and decision audit trail")
print("  ✓ Results archiving and comparison")
print("  ✓ Version tracking for audit trail")
print("  ✓ Exact reproduction of this backtest")
print("  ✓ Traceable performance metrics with run_id")
if using_bloomberg:
    print("  ✓ Bloomberg Terminal data provenance")

2025-11-07 22:49:22,014 - aponyx.persistence.json_io - INFO - Saving JSON to data\processed\run_metadata.json (10 top-level keys)


Governance Metadata Saved:
  Run ID: end_to_end_demo_20251107_224922
  Framework version: 0.1.4
  Data source: Synthetic data
  Timestamp: 2025-11-07T22:49:22.014315
  Metadata file: data\processed\run_metadata.json

Reproducibility Information:
  Signal config: lookback=20, min_periods=10
  Evaluation config: lags=[1, 3, 5], min_obs=100
  Data: 252 days starting 2023-01-01
  Synthetic data seed: 42
  Thresholds: entry=1.5, exit=0.75

Evaluation Results (included in metadata):
  Composite Score: 0.964
  Decision: PASS

Performance Metrics (included in metadata):
  Sharpe Ratio: -0.422
  Total Return: $-1,023,476.72
  Max Drawdown: $-6,394,322.82
  Number of Trades: 15
  Hit Rate: 33.3%

This metadata enables:
  ✓ Configuration documentation
  ✓ Evaluation tracking and decision audit trail
  ✓ Results archiving and comparison
  ✓ Version tracking for audit trail
  ✓ Exact reproduction of this backtest
  ✓ Traceable performance metrics with run_id


: 

## Summary

This notebook demonstrated the complete end-to-end workflow with registry-based governance:

1. Fetched market data from Bloomberg Terminal (or synthetic fallback)
2. Computed signals via SignalRegistry (batch computation)
3. **Evaluated signal-product suitability (pre-backtest quality gate)**
4. Executed systematic backtest with risk controls (conditional on evaluation)
5. Calculated performance metrics
6. Visualized results with interactive plots
7. Exported results for persistence
8. Tracked metadata with version info, evaluation results, and data source for reproducibility

### Evaluation Framework Highlights

The **signal suitability evaluation** acts as a quality gate before backtesting:

- **Four-component scoring**: Data health, predictive power, economic significance, stability
- **Decision thresholds**: PASS (≥0.7), HOLD (0.4-0.7), FAIL (<0.4)
- **Automated reports**: Markdown reports saved to `reports/suitability/`
- **Prevents wasted computation**: Low-quality signals are archived without backtesting
- **Reproducible decisions**: Evaluation config and results tracked in metadata

### Next Steps

- Evaluate other signals independently (cdx_vix_gap, spread_momentum)
- Compare evaluation scores across different signal-product pairs
- Test different strategies via StrategyRegistry (conservative, aggressive)
- Adjust position sizing and risk limits in `BacktestConfig`
- Test different CDX indices (HY, XO) and tenors
- Extend to multi-tenor strategies
- Use Bloomberg Terminal for production backtests with real data

### Key Takeaways

- **Bloomberg integration** with graceful fallback to synthetic data
- **Registry pattern** enables centralized governance and reproducibility
- **Evaluation layer** prevents backtesting low-quality signals (saves time and computation)
- All components are **modular** and **composable**
- Data generation is **deterministic** (reproducible with seeds for synthetic)
- Signals follow **consistent sign convention** (positive = long credit)
- **Metadata tracking** with version info, evaluation results, and data source ensures reproducibility
- Visualization provides **actionable insights** into strategy behavior
- Framework supports **rapid iteration** on investment ideas
- Same API works for both Bloomberg and synthetic data sources
- Each signal evaluated independently for clear attribution
- **Quality gates** ensure only suitable signals proceed to backtest